In [ ]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib notebook

# Load MNIST dataset

In [ ]:
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()

# Normalize values to [0, 1]
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Encoder and decoder architecture
We use a convolutional architecture for both the encoder and the decoder. What do we change in the variational version?

- Note1: predicting a full covariance matrix has a cost of $\mathcal{O}(D^2)$ (where D is the number of dimensions). Although in this toy example we only use two dimensions for visualization purposes, in practice tens or hundreds of dimensions are used, which leads to an unreasonable number of model parameters. For this reason, the common practice is to predict a diagonal covariance matrix, which can be compactly represented as a vector.
- Note2: to enforce $\sigma > 0$, we predict $logvar = \log \sigma^2$ which is defined everywhere, and compute $\sigma = \exp(logvar/2)$ when needed. This representation also fits well with the expression for the KL divergence.

In [ ]:
def construct_encoder(n_dimensions=2, variational=False):
    # Input: 28x28 images with only 1 color channel (since MNIST is black and white)
    X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
    
    # Two convolutional layers
    conv1 = tf.layers.conv2d(X, filters=32, kernel_size=5, strides=(2, 2), padding='same', activation=tf.nn.relu)
    conv2 = tf.layers.conv2d(conv1, filters=32, kernel_size=3, strides=(2, 2), padding='same', activation=tf.nn.relu)
    
    # Two fully-connected layers
    flattened = tf.layers.flatten(conv2)
    dense1 = tf.layers.dense(flattened, units=128, activation=tf.nn.relu)
    Z_mu = tf.layers.dense(dense1, units=n_dimensions) # No activation
    
    if variational:
        # Variational autoencoder with soft embeddings
        # To avoid introducing a constraint sigma > 0, we return log(sigma^2) instead
        Z_logvar = tf.layers.dense(dense1, units=n_dimensions) # No activation
        
        # Return input placeholder X and output functions Z_mu and Z_logvar
        return X, Z_mu, Z_logvar
    else:
        # Traditional non-linear autoencoder with point embeddings
        # Return input placeholder X and output function Z
        return X, Z_mu

In [ ]:
def construct_decoder(Z, n_dimensions=2):
    # Two fully-connected layers
    dense1 = tf.layers.dense(Z, units=128, activation=tf.nn.relu)
    dense2 = tf.layers.dense(dense1, units=7*7*32, activation=tf.nn.relu)
    
    # Two deconvolutional layers (i.e. transposed convolutions)
    reshaped = tf.reshape(dense2, [-1, 7, 7, 32])
    deconv1 = tf.layers.conv2d_transpose(reshaped, filters=32, kernel_size=3, strides=(2, 2),
                                         padding='same', activation=tf.nn.relu)
    X_reconstruction = tf.layers.conv2d_transpose(deconv1, filters=1, kernel_size=5, strides=(2, 2),
                                                  padding='same') # No activation
    
    return X_reconstruction

In [ ]:
def log_likelihood_loss(X_reconstruction, X_target):
    # TODO: complete this part
    loss = # ...
    
    return loss

In [ ]:
# TODO: define the TensorFlow graph here
init = tf.global_variables_initializer()

In [ ]:
session = tf.Session()
session.run(init)

batch_size = 256

iteration = 0
for epoch in range(50):
    perm_indices = np.random.permutation(X_train.shape[0])
    
    index = 0
    
    total_loss = 0
    N_batches = 0
    while index < X_train.shape[0]:
        X_batch = X_train[index:index+batch_size]
        index += batch_size
        
        # TODO: Complete the training code here
        loss_value = # ...
        
        total_loss += loss_value
        N_batches += 1
        iteration += 1
        
    total_loss /= N_batches
    
    print('[Epoch {}] bce_loss: {}'.format(epoch+1, total_loss))

# Sample reconstruction from the test set (AE)

In [ ]:
# Draw a sample (256 random digits organized in a 16x16 grid) from the test set
X_sample = X_test[np.random.choice(X_test.shape[0], size=16*16)]

# Compute the reconstruction for those digits
recon_viz = # ...

In [ ]:
# Show the digits in a 16x16 grid

plt.figure(figsize=(10,10))
plt.imshow(X_sample.reshape(16, 16, 28, 28).transpose(0, 2, 1, 3).reshape(16*28, 16*28))

plt.figure(figsize=(10,10))
plt.imshow(recon_viz.reshape(16, 16, 28, 28).transpose(0, 2, 1, 3).reshape(16*28, 16*28))

# Visualize the latent space (AE)

In [ ]:
# Compute the latent codes for all MNIST test digits
Z_test = # ...

In [ ]:
# Plot their 2-dimensional embeddings and color them according to their class
plt.figure(figsize=(8, 8))
plt.scatter(*Z_test.T, 0.8, c=Y_test, cmap=plt.cm.get_cmap('Paired', 10))
plt.colorbar()

In [ ]:
def reparameterize(Z_mu, Z_logvar):
    # Return a sample from a Gaussian distribution with mean "mu" and diagonal log-variance "logvar"
    # Note: logvar is not a full covariance matrix,
    # but a vector whose values correspond to the diagonal of the covariance matrix.
    
    # Return a sample from Z
    Z_sample = # ...
    
    return Z_sample

def kld_loss(Z_mu, Z_logvar):
    # Return the KL divergence between Z and a Gaussian prior N(0, I)
    kld = # ...
    return kld

In [ ]:
tf.reset_default_graph()

# TODO: define the TensorFlow graph here
init = tf.global_variables_initializer()

In [ ]:
session = tf.Session()
session.run(init)

batch_size = 256

iteration = 0
for epoch in range(50):
    perm_indices = np.random.permutation(X_train.shape[0])
    
    index = 0
    
    total_loss = 0
    total_kld = 0
    N_batches = 0
    while index < X_train.shape[0]:
        X_batch = X_train[index:index+batch_size]
        index += batch_size
        
        # TODO: Complete the training code here
        loss_value = # ...
        kld_value = # ...
        
        total_loss += loss_value
        total_kld += kld_value
        N_batches += 1
        iteration += 1
        
    total_loss /= N_batches
    total_kld /= N_batches
    
    print('[Epoch {}] bce_loss: {} - kld_loss: {}'.format(epoch+1, total_loss, total_kld))

# Sample reconstruction from the test set (VAE)

In [ ]:
# Draw a sample (256 random digits organized in a 16x16 grid) from the test set
X_sample = X_test[np.random.choice(X_test.shape[0], size=16*16)]

# Compute the reconstruction for those digits
recon_viz = # ...

In [ ]:
# Show the digits in a 16x16 grid

plt.figure(figsize=(10,10))
plt.imshow(X_sample.reshape(16, 16, 28, 28).transpose(0, 2, 1, 3).reshape(16*28, 16*28))

plt.figure(figsize=(10,10))
plt.imshow(recon_viz.reshape(16, 16, 28, 28).transpose(0, 2, 1, 3).reshape(16*28, 16*28))

# Visualize the latent space (VAE)
This code features an interactive visualization of the generated digits as you move the mouse across the latent space.

In [ ]:
# Compute the latent codes for all MNIST test digits
Z_test = # ...

In [ ]:
%matplotlib notebook
rendered_digit = np.zeros((28, 28))

plt.figure(figsize=(8, 8))
plt.scatter(*Z_test.T, 0.8, c=Y_test, cmap=plt.cm.get_cmap('Paired', 10))
plt.gca().set_aspect('equal')
plt.colorbar()
def mouse_move(event):
    global digit
    
    if event.xdata is not None:
        embedding = np.array([[event.xdata, event.ydata]], dtype='float32')
        
        # Generate with Z = embedding
        rendered_digit = # ...
        
        rendered_digit = rendered_digit.reshape(28, 28)
        digit.set_data(rendered_digit)
    
plt.connect('motion_notify_event', mouse_move)
plt.show()

plt.figure(figsize=(4, 4))
digit = plt.imshow(rendered_digit, vmin=0, vmax=1, interpolation='bilinear')

# Sample from the generative model

In [ ]:
Z_sample = # ...
recon_viz = # ...

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(recon_viz.reshape(16, 16, 28, 28).transpose(0, 2, 1, 3).reshape(16*28, 16*28))